In [1]:
from face_recog import Face_recog
from face_land_mp import face_landmarks_mp
from headpose import head_main
from eye_tracker import eye_tracking
from imutils import face_utils
import time
import matplotlib.pyplot as plt
import numpy as np
import mediapipe as mp
import cv2
import dlib

In [2]:
def print_fps(frame):
    global pTime
    cTime = time.time()
    fps = 1/(cTime - pTime)
    pTime = cTime
    cv2.putText(frame, f"FPS: {int(fps)}", (60, 70), font, 3, (0,255,0), 2)
    return frame

In [3]:
# Crop face based on its bounding box
def get_face(frame, bbox):
    real_h, real_w, c = frame.shape
    x,y,w,h = bbox
    y1 = 0 if y < 0 else y
    x1 = 0 if x < 0 else x 
    y2 = real_h if y1 + h > real_h else y + h
    x2 = real_w if x1 + w > real_w else x + w
    face = frame[y1:y2,x1:x2,:]
    return face

In [4]:
def face_detection(frame_in, draw = True, score= False, confidence = 0.85, copy=False):
    """
    Outputs the frame with detected face, alert_bool and cropped face
    """
    global noface_count
    global multiple_faces_count
    alert_bool = False
    bboxes = []
    mp_face_detection = mp.solutions.face_detection
    mp_drawing = mp.solutions.drawing_utils
    if copy:
        frame = frame_in.copy()
    else:
        frame = frame_in
    
    
    with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence= confidence) as face_detector:
        
        # To improve performance, optionally mark the frame as not writeable to
        # pass by reference.
        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Face detection:
        results = face_detector.process(frame)
        frame.flags.writeable = True
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            
    # Absence of any face detection
    if not results.detections:
        noface_count+=1
        alert_bool = True
        cv2.putText(frame, 'Alert! No faces detected for '+str(noface_count)+' times', (30, 30), font, 1, (0, 255, 255), 2)
    # Multiple faces detection
    elif len(results.detections)>1:
        multiple_faces_count += 1
        alert_bool = True
        cv2.putText(frame, 'Alert! multiple faces detected for '+str(multiple_faces_count)+' times', (30, 30), font, 1, (0, 255, 255), 2)
    
    # Get bboxes for detected faces and draw the face detection annotations on the frame.
    if results.detections:
        for id, detection in enumerate(results.detections):
            bbox = detection.location_data.relative_bounding_box
            ih, iw, ic = frame.shape
            bbox = int(bbox.xmin * iw), int(bbox.ymin * ih), int(bbox.width * iw), int(bbox.height * ih)
            bboxes.append(bbox)
            if draw:
                cv2.rectangle(frame, bbox, (255, 0, 255), 2)
                if score:
                    cv2.putText(frame, f'{int(detection.score[0] * 100)}%', (bbox[0], bbox[1] - 20), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2)
    
    return alert_bool, frame, bboxes

In [5]:
def face_recognition(frame, fr, bbox):
    global failed_verif_count
    
    # Detect and recognise Faces
    face_locations, face_names = fr.detect_known_faces(frame)
    
    alert_bool = not face_names
    #Draw box and name
    for face_loc, name in zip(face_locations, face_names):
        cv2.putText(frame, name, (bbox[0], bbox[1] - 20), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2)
        
    if not alert_bool:
        cv2.putText(frame, 'You are verified', (30, 30), font, 1, (0, 255, 255), 2)
    else:
        failed_verif_count += 1
        cv2.putText(frame, 'Alert! You are not the actual user: '+str(failed_verif_count), (30, 30), font, 1, (0, 255, 255), 2)

In [6]:
def facial_landmarks_detection(frame_in, draw= True, copy=False, module = "Dlib"):
    if copy:
        frame = frame_in.copy()
    else:
        frame = frame_in
    
    if module == "Dlib":
        # detect faces in the grayscale image
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)   
        rects = detector(gray, 0)
        ret = False
        shape = ""
    
        # loop over the face detections
        for (i, rect) in enumerate(rects):
            # determine the facial landmarks for the face region, then
            # convert the facial landmark (x, y)-coordinates to a NumPy
            # array
            ret = True
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
        
            # loop over the (x, y)-coordinates for the facial landmarks
            # and draw them on the frame
            if draw:
                for (x, y) in shape:
                    cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
                    
        return ret, shape, frame
    
    elif module == "mediapipe":
        alert_bool, shape, frame = face_landmarks_mp(frame)
        return False, shape, frame

In [7]:
def capture_image():
    while True:
        blank = cv2.imread('db/blank.png')
        cv2.putText(blank, 'press r to capture image', (30, 30), font, 1, (0, 0, 255), 2)
        cv2.imshow("Output", blank)
        if cv2.waitKey(1) & 0xFF == ord('r'):
            break
    cv2.destroyAllWindows()

    # capturing image
    webcam = cv2.VideoCapture(0) 
    ret, frame = webcam.read()
    
    # saving image as use_image.jpg
    # for further face verification
    cv2.imwrite("captures/user_image.jpg", frame)
    webcam.release()
    cv2.destroyAllWindows()
    
    # reading picture as user_pic
    user_pic = cv2.imread('captures/user_image.jpg')

In [8]:
font = cv2.FONT_HERSHEY_PLAIN 

# capture user image
capture_image()

# Face recognizer
fr = Face_recog()
fr.load_encoding_images("captures/")

# Facial landmarks predictor
saved_model = "models/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(saved_model)

noface_count = 0
multiple_faces_count = 0
failed_verif_count = 0
pTime = 0

1 encoding images found.
Encoding images loaded


In [9]:
def main():
    cap = cv2.VideoCapture(0)
    cv2.namedWindow('PROCTORING ON')
    while(True):
        # Video capture frame by frame
        ret, frame = cap.read()
        frame = print_fps(cv2.flip(frame, 1))
        
        #Faces detection
        alert_bool, frame, bboxes =  face_detection(frame, confidence = 0.95, copy=False)
        
        #Only if single face detected
        if not alert_bool:    
            
            #Face verification
            face_recognition(frame, fr, bboxes[0])
            
            # Facial landmarks detection
            ret, shape, frame = facial_landmarks_detection(frame, copy=False, module="Dlib", draw= True)
            if ret:
                head_main(frame,shape, copy=False)
                eye_tracking(frame, shape, threshold = 75)
        
        cv2.imshow('PROCTORING ON',  frame)
                
        if cv2.waitKey(1) & 0xFF == 27: 
            break
    cap.release()
    cv2.destroyAllWindows()
    return

In [10]:
if __name__ == "__main__":
    main()
    exit()

C:\Users\Anirudh\mini_project_iiita\headpose.py:134: RuntimeWarning: divide by zero encountered in int_scalars
  m = (x2[1] - x1[1])/(x2[0] - x1[0])


In [11]:
#    DO NOT DELETE THIS!
# Warnings in head pose estimation:
#     C:\Users\Anirudh\mini_project_iiita\headpose.py:129: RuntimeWarning: divide by zero encountered in int_scalars
#   m = (x2[1] - x1[1])/(x2[0] - x1[0])
#   ang2 = ...(1/m)..
# C:\Users\Anirudh\mini_project_iiita\eye_tracker.py:39: RuntimeWarning: divide by zero encountered in long_scalars
#   y_ratio = (cy - end_points[1])/(end_points[3] - cy)

In [12]:
# Rough:
        # outputs: detreg_out, landeye_out, head_out
#         head_out = cv2.copyMakeBorder(head_out, 0, 0, 320, 320, cv2.BORDER_CONSTANT, (0,0,0))
#         horiz = np.concatenate((horiz, head_out), axis = 1)